In [1]:
import realtabformer_generalized

# Single Table Test

In [3]:
import pandas as pd
import numpy as np

np.random.seed(42)

df = pd.DataFrame(np.random.randn(100, 3), columns=['A', 'B', 'C'])

llm_name = "meta-llama/Llama-3.2-1B"

rtf_model = realtabformer_generalized.REaLTabFormer(
    model_type = "tabular",
    llm_name = llm_name,
    gradient_accumulation_steps = 4,
    logging_steps=100,
    epochs=10, batch_size = 5, train_size = 0.8,
    checkpoints_dir = 'D:/LLMs',
    samples_save_dir = 'D:/LLMs') # Please change this to your own drive with sufficient storage. 

rtf_model.fit(df)

rtf_model.save("rtf_model/")

samples = rtf_model.sample(n_samples=len(df))

Computing the sensitivity threshold...
Using parallel computation!!!


C:\Users\thoma\Downloads\realtabformer_generalized\realtabformer.py:601: UserWarning: Duplicate rate (0.0) in the data is zero. The `qt_interval` will be set                     to qt_interval_unique=100.
  warnings.warn(
C:\Users\thoma\Downloads\realtabformer_generalized\realtabformer.py:628: UserWarning: qt_interval adjusted from 100 to 2...
  warnings.warn(


Bootstrap round:   0%|          | 0/500 [00:00<?, ?it/s]

Sensitivity threshold summary:
count    500.000000
mean       0.018287
std        0.037938
min       -0.025000
25%       -0.012500
50%        0.012500
75%        0.037500
max        0.181250
dtype: float64
Sensitivity threshold: 0.09375 qt_max: 0.05


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

...

Step,Training Loss,Validation Loss


  0%|          | 0/48 [00:00<?, ?it/s]

Generated 0 invalid samples out of total 128 samples generated. Sampling efficiency is: 100.0000%
Saving not-best model...
Critic round: 10,                     sensitivity_threshold: 0.09375,                         val_sensitivity: 0.17041666666666666,                             val_sensitivities: [0.15625, 0.20625, 0.10625000000000001, 0.21875, 0.11875, 0.1875, 0.11249999999999999, 0.14375, 0.2, 0.10625, 0.21875, 0.1875, 0.2125, 0.23125, 0.15000000000000002]
Copying artefacts from: best-disc-model
Copying artefacts from: mean-best-disc-model
Copying artefacts from: not-best-disc-model
Copying artefacts from: last-epoch-model


  0%|          | 0/100 [00:00<?, ?it/s]

Generated 0 invalid samples out of total 128 samples generated. Sampling efficiency is: 100.0000%


# Multi Table Test

In [1]:
from sdv.datasets.demo import download_demo
import os
import pandas as pd
import numpy as np
import realtabformer_generalized
from pathlib import Path

real_data, metadata = download_demo(
    modality='multi_table',
    dataset_name='fake_hotels'
)

child = real_data['guests']
parent = real_data['hotels']

parent = parent.fillna(5)
child = child.dropna()

join_on = "hotel_id"

assert ((join_on in parent.columns) and
        (join_on in child.columns))

llm_name = "meta-llama/Llama-3.2-1B"

rtf_parent = realtabformer_generalized.REaLTabFormer(
    model_type = "tabular",
    llm_name = llm_name,
    gradient_accumulation_steps = 4,
    logging_steps=100,
    epochs = 10, batch_size = 5, train_size = 0.8,
    checkpoints_dir = 'D:/LLMs',
    samples_save_dir = 'D:/LLMs')

rtf_parent.fit(parent.drop(join_on, axis=1))

pdir = Path("rtf_parent/") #Also save it somewhere with plenty of space。
rtf_parent.save(pdir)
parent_model_path = sorted([p for p in pdir.glob("id*") if p.is_dir()], key=os.path.getmtime)[-1]

child_model_1 = realtabformer_generalized.REaLTabFormer(model_type="relational",
                    parent_realtabformer_path=parent_model_path, epochs = 1, batch_size = 1, train_size = 0.8,
                             checkpoints_dir = 'D:/LLMs',
                            samples_save_dir = 'D:/LLMs')

Computing the sensitivity threshold...
Using parallel computation!!!


C:\Users\thoma\Downloads\realtabformer_generalized\realtabformer.py:601: UserWarning: Duplicate rate (0.0) in the data is zero. The `qt_interval` will be set                     to qt_interval_unique=100.
  warnings.warn(
C:\Users\thoma\Downloads\realtabformer_generalized\realtabformer.py:628: UserWarning: qt_interval adjusted from 100 to 2...
  warnings.warn(


Bootstrap round:   0%|          | 0/500 [00:00<?, ?it/s]

Sensitivity threshold summary:
count    500.000000
mean       0.231800
std        0.254762
min       -0.025000
25%       -0.025000
50%        0.375000
75%        0.375000
max        0.975000
dtype: float64
Sensitivity threshold: 0.575 qt_max: 0.05


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

...

C:\Users\thoma\anaconda3\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['lm_head.weight'].
C:\Users\thoma\anaconda3\Lib\site-packages\transformers\trainer.py:3354: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user 

Step,Training Loss,Validation Loss


  0%|          | 0/3 [00:00<?, ?it/s]

Generated 0 invalid samples out of total 128 samples generated. Sampling efficiency is: 100.0000%
Critic round: 10,                     sensitivity_threshold: 0.575,                         val_sensitivity: 0.02833333333333333,                             val_sensitivities: [-0.025, -0.025, -0.025, -0.025, -0.025, -0.025, 0.375, -0.025, -0.025, 0.375, -0.025, -0.025, -0.025, -0.025, -0.025]
Copying artefacts from: best-disc-model
Copying artefacts from: mean-best-disc-model
Copying artefacts from: not-best-disc-model
Copying artefacts from: last-epoch-model


C:\Users\thoma\Downloads\realtabformer_generalized\realtabformer.py:194: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(parent_realtabformer_path / ModelFileName.r

In [2]:
child_model_1.fit(df = child,
                    in_df = parent,
                    join_on = join_on, num_bootstrap = 1)

C:\Users\thoma\Downloads\realtabformer_generalized\data_utils.py:152: UserWarning: Default values will be overridden because transform_data was passed...
  warnings.warn(


Map:   0%|          | 0/579 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

LlamaConfig {
  "_name_or_path": "meta-llama/Llama-3.2-1B",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 5,
  "eos_token_id": 6,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 6,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 32.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.46.3",
  "use_cache": true,
  "vocab_size": 35
}

GPT2Config {
  "activation_function": "gelu_new",
  "attn_

Config of the encoder: <class 'transformers.models.llama.modeling_llama.LlamaModel'> is overwritten by shared encoder config: LlamaConfig {
  "_name_or_path": "meta-llama/Llama-3.2-1B",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 5,
  "eos_token_id": 6,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 6,
  "num_key_value_heads": 8,
  "pad_token_id": 2,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 32.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_word_embeddings": true,
  "torch_dtype": "b

AssertionError: There should be no missing_keys after loading the pretrained GPT2 state!

In [ ]:
parent_samples = parent_model.sample(parent_n)
    parent_samples.index.name = join_on
    parent_samples = parent_samples.reset_index()
    
    child_samples = child_model_1.sample(n_samples = child_n,
                    input_unique_ids=parent_samples[join_on],
                    input_df=parent_samples.drop(join_on, axis=1),
                    output_max_length = None,
                    gen_batch = 1)
        
    child_samples.index.name = join_on